$
% Define TeX macros for this document
\def\vv#1{\boldsymbol{#1}}
\def\mm#1{\boldsymbol{#1}}
\def\R#1{\mathbb{R}^{#1}}
\def\SO{SO(3)}
\def\triad{\mm{\Lambda}}
$

# Discretization of rotation field

## Direct interpolation of triads

In the following example, we want to show that a direct interpolation of the rotation field using standard Lagrange shape functions does not lead to a valid rotation field.

In [ ]:
import numpy as np
from beamme.core.rotation import Rotation

np.set_printoptions(suppress=True)

rot_1 = Rotation([0, 1, 0], np.pi / 2)
rot_2 = Rotation([1, 0, 0], np.pi / 2)

R_1 = rot_1.get_rotation_matrix()
R_2 = rot_2.get_rotation_matrix()

print(f"Rotation matrix 1:\n{R_1}")
print(f"Rotation matrix 2:\n{R_2}")

R_interpolated = R_1 * 0.5 + R_2 * 0.5
print(f"Interpolated rotation matrix:\n{R_interpolated}")
print(f"Determinant of interpolated rotation matrix: {np.linalg.det(R_interpolated)}")

## Interpolation of total rotation vectors

In this example, we will show that a direct interpolation of the total rotation vectors also does not lead to an objective rotation field.
We will demonstrate this by applying a rigid body rotation and showing that the interpolated rotation field changes.

In [ ]:
import numpy as np
from beamme.core.rotation import Rotation


def total_rotation_vector_interpolation(rot_1, rot_2, xi):
    """Perform an interpolation between the two given rotations based on the total rotation vectors."""

    rot_vec_1 = rot_1.get_rotation_vector()
    rot_vec_2 = rot_2.get_rotation_vector()

    rot_vec_interpolated = rot_vec_1 * 0.5 * (1 - xi) + rot_vec_2 * 0.5 * (1 + xi)

    return Rotation.from_rotation_vector(rot_vec_interpolated)


# Perform the interpolation at xi = 0.0
rot_1 = Rotation([0, 1, 0], np.pi / 2)
rot_2 = Rotation([1, 0, 0], np.pi / 2)
interpolated_rot = total_rotation_vector_interpolation(rot_1, rot_2, 0.0)
print(f"Original interpolated rotation: {interpolated_rot.get_rotation_vector()}")

# Apply an arbitrary rigid body rotation to the two rotations and perform the interpolation again
rot_rigid_body = Rotation([1, 2, 3], np.pi / 3)
rot_1_rotated = rot_rigid_body * rot_1
rot_2_rotated = rot_rigid_body * rot_2
interpolated_rot_with_rigid_body_rotation = total_rotation_vector_interpolation(
    rot_1_rotated, rot_2_rotated, 0.0
)
print(
    f"Interpolated rotation with rigid body rotation: {interpolated_rot_with_rigid_body_rotation.get_rotation_vector()}"
)

# Show that the interpolated rotation with rigid body rotation is not equal to the rigid body rotation applied to the interpolated rotation
print(
    f"Rigid body rotation applied to original interpolated rotation: {(rot_rigid_body * interpolated_rot).get_rotation_vector()}"
)

## Interpolation of relative rotation vectors

In this example, we will show that the interpolation of relative rotation vectors leads to an objective interpolation.

In [ ]:
import numpy as np
from beamme.core.rotation import Rotation


def relative_rotation_vector_interpolation(rot_ref, rot_1, rot_2, xi):
    """Perform an interpolation between the two given rotations based on relative rotation vectors."""

    relative_rotation_1 = rot_ref.inv() * rot_1
    relative_rotation_2 = rot_ref.inv() * rot_2

    relative_rotation_1_vector = relative_rotation_1.get_rotation_vector()
    relative_rotation_2_vector = relative_rotation_2.get_rotation_vector()

    relative_rotation_vector_interpolated = relative_rotation_1_vector * 0.5 * (
        1 - xi
    ) + relative_rotation_2_vector * 0.5 * (1 + xi)

    interpolated_rotation = rot_ref * Rotation.from_rotation_vector(
        relative_rotation_vector_interpolated
    )
    return interpolated_rotation


# Perform the interpolation at xi = 0.0
rot_1 = Rotation([0, 1, 0], np.pi / 2)
rot_2 = Rotation([1, 0, 0], np.pi / 2)
rot_ref = rot_1
interpolated_rot = relative_rotation_vector_interpolation(rot_ref, rot_1, rot_2, 0.0)
print(f"Original interpolated rotation: {interpolated_rot.get_rotation_vector()}")

# Apply an arbitrary rigid body rotation to the two rotations and perform the interpolation again
rot_rigid_body = Rotation([1, 2, 3], np.pi / 3)
rot_1_rotated = rot_rigid_body * rot_1
rot_2_rotated = rot_rigid_body * rot_2
rot_ref_rotated = rot_1_rotated
interpolated_rot_with_rigid_body_rotation = relative_rotation_vector_interpolation(
    rot_ref_rotated, rot_1_rotated, rot_2_rotated, 0.0
)
print(
    f"Interpolated rotation with rigid body rotation: {interpolated_rot_with_rigid_body_rotation.get_rotation_vector()}"
)

# Show that the interpolated rotation with rigid body rotation is not equal to the rigid body rotation applied to the interpolated rotation
print(
    f"Rigid body rotation applied to original interpolated rotation: {(rot_rigid_body * interpolated_rot).get_rotation_vector()}"
)